### **imports**

In [87]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

### **making a table of all of the stars in the txt file**

In [88]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [89]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [90]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [91]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [92]:
folder_path_csv = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"
folder_path_txt = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/txtfiles"

In [93]:
num_iterations = 5
empty_list = []
for i in range(num_iterations):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path_csv, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.97


######### Lightcurve features #########

amplitude = 0.3432799329059622
cusum = 0.1452776746557756
eta = 2.5032121771222386
hl_amp_ratio = 0.3752540817315546
kurtosis = -0.8239731458557591
n_points = 548
period = 0.7067830026597548
period_SNR = 136.323076028037
period_log10FAP = -79.52952372617999
period_uncertainty = 0.00025304222789990227
phase_cusum = 0.3270529745835165
phase_eta = 0.3073737948909103
phi21 = 0.7616105986457724
phi31 = 4.672091901915031
quartile31 = 0.5005662300894382
r21 = 0.36856251711608334
r31 = 0.2740004452797228
shapiro_w = 0.8901732563972473
skewness = -0.6636568203032644
slope_per10 = -0.08311948503485267
slope_per90 = 0.12945126232614182
stetson_k = 0.7491324538122004
weighted_mean = 13.092348996917007
weighted_std = 0.28710942582204807
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.1232423590727151
cusum = 0.07297790512002132
eta = 1.950067932686292
hl_amp_ratio = 0.528598428925746
kurtosis = -0.15884420656422904
n_points = 714
period = 0.4850785323756956
period_SNR = 156.13226520718797
period_log10FAP = -81.11474893593616
period_uncertainty = 0.00010147379267899659
phase_cusum = 0.21635598963561906
phase_eta = 0.8528476553749813
phi21 = -2.5746686059722883
phi31 = -1.8861333979157895
quartile31 = 0.1559388913663824
r21 = 0.36476704576255525
r31 = 0.14180897051166658
shapiro_w = 0.9504188895225525
skewness = -0.7122921248506956
slope_per10 = -0.05250410427320134
slope_per90 = 0.05578336530605501
stetson_k = 0.7341563301505658
weighted_mean = 13.921673855963016
weighted_std = 0.1110267747987192
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.19424881679051437
cusum = 0.10203171570624923
eta = 2.1385363382664044
hl_amp_ratio = 0.34783602651267503
kurtosis = -0.425316126805952
n_points = 1345
period = 0.5946316313563619
period_SNR = 190.3815055463475
period_log10FAP = -154.61411390613216
period_uncertainty = 0.00012776786800899886
phase_cusum = 0.22778930507736841
phase_eta = 0.6449250545463119
phi21 = 0.6221185914306218
phi31 = 1.5312682374527329
quartile31 = 0.272544941719552
r21 = 0.4554864187272973
r31 = 0.21240426867259737
shapiro_w = 0.917834460735321
skewness = -0.7488185263220661
slope_per10 = -0.038623994788294834
slope_per90 = 0.03716606990882367
stetson_k = 0.690935119329278
weighted_mean = 14.789870649314969
weighted_std = 0.18273211269978998
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.3004413947793003
cusum = 0.1579172765827649
eta = 1.9304232673660537
hl_amp_ratio = 0.4030983101451894
kurtosis = -0.9110018909155841
n_points = 1277
period = 0.5624707955811346
period_SNR = 194.21730612171257
period_log10FAP = -192.36346740342373
period_uncertainty = 0.0001373219797624614
phase_cusum = 0.32607715405769583
phase_eta = 0.1226486584230441
phi21 = -2.3955298731269594
phi31 = -4.385137668990532
quartile31 = 0.4299983326988741
r21 = 0.47569186043605977
r31 = 0.2972135341682631
shapiro_w = 0.9123263359069824
skewness = -0.6012567716176801
slope_per10 = -0.05503448590386949
slope_per90 = 0.044849735211121525
stetson_k = 0.7195868431584528
weighted_mean = 14.439427666940363
weighted_std = 0.26253166278345197


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
